In [205]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable, log, exp
from biogeme import models
from biogeme import results as res

In [206]:
#data_file = "http://transp-or.epfl.ch/data/lpmc.dat"
data_file='lpmc10.dat'
lpmc = pd.read_csv(data_file, sep='\t')
lpmc

trip_id  household_id  person_n  trip_n  travel_mode  purpose  fueltype  \
0          20             5         1       0            4        3         1   
1          41             9         3       0            4        3         1   
2          69            13         2       1            4        3         1   
3         102            20         2       0            2        3         1   
4         105            21         0       1            4        3         1   
...       ...           ...       ...     ...          ...      ...       ...   
4995    80998         17591         0       5            4        3         1   
4996    81000         17592         0       0            3        3         6   
4997    81015         17597         0       3            4        3         1   
4998    81041         17604         2       4            3        1         1   
4999    81062         17607         0       4            3        5         6   

      faretype  bus_scale  survey_year  ...  dur_pt_access  dur_pt_rail  \
0            5        0.0            1  ...       0.381667     0.000000   
1            5        0.0            1  ...       0.146944     0.000000   
2            1        1.0            1  ...       0.029444     0.083333   
3            1        1.0            1  ...       0.339722     0.183333   
4            1        1.0            1  ...       0.126389     0.000000   
...        ...        ...          ...  ...            ...          ...   
4995         5        0.0            3  ...       0.189167     0.000000   
4996         5        0.0            3  ...       0.105278     0.000000   
4997         5        0.0            3  ...       0.343056     0.000000   
4998         2        0.0            3  ...       0.344444     0.316667   
4999         1        1.0            3  ...       0.069444     0.000000   

      dur_pt_bus  dur_pt_int  pt_interchanges  dur_driving  cost_transit  \
0       0.062222    0.000000                0     0.117222          0.00   
1       0.225000    0.000000                0     0.200833          0.00   
2       0.735833    0.398056                3     0.716944          6.00   
3       0.116667    0.266667                1     0.250833          3.00   
4       0.150000    0.000000                0     0.125833          1.50   
...          ...         ...              ...          ...           ...   
4995    0.117778    0.000000                0     0.082500          0.00   
4996    0.220278    0.000000                0     0.213611          0.00   
4997    0.177500    0.000000                0     0.189444          0.00   
4998    0.000000    0.083333                1     0.386111          1.05   
4999    0.166389    0.000000                0     0.146111          1.50   

      cost_driving_fuel  cost_driving_ccharge  driving_traffic_percent  
0                  0.41                   0.0                 0.097156  
1                  0.48                   0.0                 0.378976  
2                  2.16                   0.0                 0.582720  
3                  0.89                   0.0                 0.170543  
4                  0.37                   0.0                 0.154525  
...                 ...                   ...                      ...  
4995               0.22                  10.5                 0.239057  
4996               0.52                   0.0                 0.412224  
4997               0.76                   0.0                 0.086510  
4998               0.98                   0.0                 0.340288  
4999               0.28                   0.0                 0.653992  

[5000 rows x 32 columns]

In [207]:
database = db.Database('trips', lpmc)

In [208]:
trip_id = Variable('trip_id')
household_id = Variable('household_id')
person_n = Variable('person_n')
travel_mode = Variable('travel_mode')
purpose = Variable('purpose')
fueltype = Variable('fueltype')
faretype = Variable('faretype')
bus_scale = Variable('bus_scale')
survey_year = Variable('survey_year')
travel_year = Variable('travel_year')
travel_month = Variable('travel_month')
travel_day = Variable('travel_day')
day_of_week = Variable('day_of_week')
start_time = Variable('day_of_week')
age = Variable('age')
female = Variable('female')
driving_license = Variable('driving_license')
car_ownership = Variable('car_ownership')
distance = Variable
dur_walking = Variable('dur_walking')
dur_cycling = Variable('dur_cycling')
dur_pt_access = Variable('dur_pt_access')
dur_pt_rail = Variable('dur_pt_rail')
dur_pt_bus = Variable('dur_pt_bus')
dur_pt_int = Variable('dur_pt_int')
pt_interchanges = Variable('pt_interchanges')
dur_driving = Variable('dur_driving')
cost_transit = Variable('cost_transit')
cost_driving_fuel = Variable('cost_driving_fuel')
cost_driving_ccharge = Variable('cost_driving_ccharge')
driving_traffic_percent = Variable('driving_traffic_percent')

# Model 0

In [209]:
asc_walk = Beta('asc_walk', 0, None, None, 1)
asc_cycling = Beta('asc_cycling', 0, None, None, 0)
asc_public = Beta('asc_public', 0, None, None, 0)
asc_driving = Beta('asc_driving', 0, None, None, 0)

In [210]:
cost_driving = cost_driving_fuel + cost_driving_ccharge
dur_public = dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int #total duration of public transportation

In [211]:
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

In [212]:
V_walk = asc_walk + beta_time * dur_walking
V_cycling = asc_cycling + beta_time * dur_cycling
V_driving = asc_driving + beta_time * dur_driving + beta_cost * cost_driving
V_public = asc_public + beta_time * dur_public + beta_cost * cost_transit

In [213]:
'''prob_walk = 1 / (1 + exp(V_cycling - V_walk) + exp(V_driving - V_walk) + exp(V_public - V_walk))
prob_cycling = 1 / (1 + exp(V_walk - V_cycling) + exp(V_driving - V_cycling) + exp(V_public - V_cycling))
prob_driving = 1 / (1 + exp(V_cycling - V_driving) + exp(V_walk - V_driving) + exp(V_public - V_driving))
prob_public = 1-prob_driving + prob_cycling + prob_walk

prob_observation = prob_walk * (travel_mode == 1) + prob_cycling * (travel_mode == 2) + prob_driving * (travel_mode == 4) + prob_public * (travel_mode == 3) 
logprob = log(prob_observation)'''

V = {1: V_walk, 2: V_cycling, 3: V_public, 4: V_driving}

logprob = models.loglogit(V, None, travel_mode)

In [214]:
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'model_base'

In [215]:
results0 = biogeme.estimate()

In [216]:
print(results0.printGeneralStatistics())

Number of estimated parameters:	5
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4722.972
Final log likelihood:	-4722.972
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00106
Akaike Information Criterion:	9455.944
Bayesian Information Criterion:	9488.53
Final gradient norm:	3.2270E-03
Nbr of threads:	4



In [217]:
results0.getEstimatedParameters()

Value  Rob. Std err  Rob. t-test  Rob. p-value
asc_cycling -3.660820      0.100795   -36.319479           0.0
asc_driving -1.335133      0.076403   -17.474928           0.0
asc_public  -0.559065      0.052620   -10.624619           0.0
beta_cost   -0.149850      0.013780   -10.874275           0.0
beta_time   -5.410243      0.188334   -28.726794           0.0

In [218]:
results0.data.htmlFileName

'model_base.html'

### Define Model 0

In [219]:
'''V_walk = beta_time * dur_walking
V_cycling = beta_time * dur_cycling
V_driving = beta_time * dur_driving + beta_cost * cost_driving
V_public = beta_time * dur_public + beta_cost * cost_transit'''

V_walk = asc_walk + beta_time * dur_walking
V_cycling = asc_cycling + beta_time * dur_cycling
V_driving = asc_driving + beta_time * dur_driving + beta_cost * cost_driving
V_public = asc_public + beta_time * dur_public + beta_cost * cost_transit

'''prob_walk = 1 / (1 + exp(V_cycling - V_walk) + exp(V_driving - V_walk) + exp(V_public - V_walk))
prob_cycling = 1 / (1 + exp(V_walk - V_cycling) + exp(V_driving - V_cycling) + exp(V_public - V_cycling))
prob_driving = 1 / (1 + exp(V_cycling - V_driving) + exp(V_walk - V_driving) + exp(V_public - V_driving))
prob_public = 1-prob_driving + prob_cycling + prob_walk

prob_observation = prob_walk * (travel_mode == 1) + prob_cycling * (travel_mode == 2) + prob_driving * (travel_mode == 4) + prob_public * (travel_mode == 3) 
logprob = log(prob_observation)'''

V = {1: V_walk, 2: V_cycling, 3: V_public, 4: V_driving}

logprob = models.loglogit(V, None, travel_mode)

biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'model_0'
results0 = biogeme.estimate()
print(results0.printGeneralStatistics())
results0.getEstimatedParameters()

Number of estimated parameters:	5
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-5015.409
Final log likelihood:	-4722.972
Likelihood ratio test for the init. model:	584.8747
Rho-square for the init. model:	0.0583
Rho-square-bar for the init. model:	0.0573
Akaike Information Criterion:	9455.944
Bayesian Information Criterion:	9488.53
Final gradient norm:	3.2304E-03
Nbr of threads:	4



Value  Rob. Std err  Rob. t-test  Rob. p-value
asc_cycling -3.660820      0.100795   -36.319479           0.0
asc_driving -1.335133      0.076403   -17.474928           0.0
asc_public  -0.559065      0.052620   -10.624619           0.0
beta_cost   -0.149850      0.013780   -10.874275           0.0
beta_time   -5.410243      0.188334   -28.726794           0.0

# Model 1


In [220]:
asc_walk = Beta('asc_walk', 0, None, None, 1)
asc_cycling = Beta('asc_cycling', 0, None, None, 0)
asc_public = Beta('asc_public', 0, None, None, 0)
asc_driving = Beta('asc_driving', 0, None, None, 0)

In [221]:
cost_driving = cost_driving_fuel + cost_driving_ccharge
dur_public = dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int #total duration of public transportation

In [222]:
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycling = Beta('beta_time_cycling', 0, None, None, 0)
beta_time_driving = Beta('beta_time_driving', 0, None, None, 0)
beta_time_public = Beta('beta_time_public', 0, None, None, 0)

In [223]:
V_walk = asc_walk + beta_time_walk * dur_walking
V_cycling = asc_cycling + beta_time_cycling * dur_cycling
V_driving = asc_driving + beta_time_driving * dur_driving + beta_cost * cost_driving
V_public = asc_public + beta_time_public * dur_public + beta_cost * cost_transit
'''V_walk = beta_time_walk * dur_walking
V_cycling =  beta_time_cycling * dur_cycling
V_driving = beta_time_driving * dur_driving + beta_cost * cost_driving
V_public = beta_time_public * dur_public + beta_cost * cost_transit'''

'V_walk = beta_time_walk * dur_walking\nV_cycling =  beta_time_cycling * dur_cycling\nV_driving = beta_time_driving * dur_driving + beta_cost * cost_driving\nV_public = beta_time_public * dur_public + beta_cost * cost_transit'

In [224]:

'''prob_walk = 1 / (1 + exp(V_cycling - V_walk) + exp(V_driving - V_walk) + exp(V_public - V_walk))
prob_cycling = 1 / (1 + exp(V_walk - V_cycling) + exp(V_driving - V_cycling) + exp(V_public - V_cycling))
prob_driving = 1 / (1 + exp(V_cycling - V_driving) + exp(V_walk - V_driving) + exp(V_public - V_driving))
prob_public = 1-prob_driving + prob_cycling + prob_walk'''

prob_observation = prob_walk * (travel_mode == 1) + prob_cycling * (travel_mode == 2) + prob_driving * (travel_mode == 4) + prob_public * (travel_mode == 3) 
logprob = log(prob_observation)

V = {1: V_walk, 2: V_cycling, 3: V_public, 4: V_driving}

logprob = models.loglogit(V, None, travel_mode)

In [225]:
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'model_1'

In [226]:
results1 = biogeme.estimate()

In [227]:
print(results1.printGeneralStatistics())

Number of estimated parameters:	8
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4412.008
Final log likelihood:	-4412.008
Likelihood ratio test for the init. model:	2.735721e-05
Rho-square for the init. model:	3.1e-09
Rho-square-bar for the init. model:	-0.00181
Akaike Information Criterion:	8840.016
Bayesian Information Criterion:	8892.154
Final gradient norm:	2.4625E-02
Nbr of threads:	4



In [228]:
results1.getEstimatedParameters()

Value  Rob. Std err  Rob. t-test  Rob. p-value
asc_cycling       -4.590559      0.179038   -25.640156           0.0
asc_driving       -2.072176      0.120171   -17.243521           0.0
asc_public        -2.439718      0.121955   -20.005013           0.0
beta_cost         -0.141794      0.015193    -9.332654           0.0
beta_time_cycling -5.195457      0.423894   -12.256490           0.0
beta_time_driving -5.874430      0.359744   -16.329457           0.0
beta_time_public  -3.199827      0.230717   -13.869059           0.0
beta_time_walk    -8.367753      0.360131   -23.235336           0.0

In [229]:
results1.data.htmlFileName

'model_1.html'

## Comparing Models 0 and 1

In [230]:
general_statistics_model_0 = results0.getGeneralStatistics()
general_statistics_model_0

{'Number of estimated parameters': GeneralStatistic(value=5, format=''),
 'Sample size': GeneralStatistic(value=5000, format=''),
 'Excluded observations': GeneralStatistic(value=0, format=''),
 'Init log likelihood': GeneralStatistic(value=-5015.409157399887, format='.7g'),
 'Final log likelihood': GeneralStatistic(value=-4722.971810901689, format='.7g'),
 'Likelihood ratio test for the init. model': GeneralStatistic(value=584.8746929963945, format='.7g'),
 'Rho-square for the init. model': GeneralStatistic(value=0.058307774564459236, format='.3g'),
 'Rho-square-bar for the init. model': GeneralStatistic(value=0.05731084692743427, format='.3g'),
 'Akaike Information Criterion': GeneralStatistic(value=9455.943621803379, format='.7g'),
 'Bayesian Information Criterion': GeneralStatistic(value=9488.52958776046, format='.7g'),
 'Final gradient norm': GeneralStatistic(value=0.00323035292982025, format='.4E'),
 'Nbr of threads': GeneralStatistic(value=4, format='')}

In [231]:
general_statistics_model_1 = results1.getGeneralStatistics()
general_statistics_model_1

{'Number of estimated parameters': GeneralStatistic(value=8, format=''),
 'Sample size': GeneralStatistic(value=5000, format=''),
 'Excluded observations': GeneralStatistic(value=0, format=''),
 'Init log likelihood': GeneralStatistic(value=-4412.007995234046, format='.7g'),
 'Final log likelihood': GeneralStatistic(value=-4412.007981555442, format='.7g'),
 'Likelihood ratio test for the init. model': GeneralStatistic(value=2.7357207727618515e-05, format='.7g'),
 'Rho-square for the init. model': GeneralStatistic(value=3.100312562231977e-09, format='.3g'),
 'Rho-square-bar for the init. model': GeneralStatistic(value=-0.001813230241204833, format='.3g'),
 'Akaike Information Criterion': GeneralStatistic(value=8840.015963110884, format='.7g'),
 'Bayesian Information Criterion': GeneralStatistic(value=8892.153508642214, format='.7g'),
 'Final gradient norm': GeneralStatistic(value=0.024625293437189763, format='.4E'),
 'Nbr of threads': GeneralStatistic(value=4, format='')}

### By hand

In [232]:
alpha = 0.05 #significance level

In [233]:
#compute the log-likelihood ratio
L0=general_statistics_model_0['Final log likelihood'].value
L1=general_statistics_model_1['Final log likelihood'].value
test = -2 * (L0 - L1)
test

621.9276586924952

In [234]:
#compute the degrees of freedom for the chi2 distribution
K0 = general_statistics_model_0['Number of estimated parameters'].value
K1 = general_statistics_model_1['Number of estimated parameters'].value
degrees_of_freedom = K1 - K0
degrees_of_freedom

3

In [235]:
from scipy.stats import chi2
threshold = chi2.ppf(1-alpha, degrees_of_freedom)
threshold

7.814727903251179

621.9276321304606 > 7.814727903251179 so we reject the null hypothesis at level 0.05%.

### Automatically

In [236]:
results1.likelihood_ratio_test(results0, alpha)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=621.9276586924952, threshold=7.814727903251179)

The null hypothesis is rejected at the 5% level.

### Model 2
include pt_interchanges for alternative public transport and driving_liscence to interact with ASC_driving

Model specification:

$ $$V_{walk}$$=ASC_{walk}+\beta_{time\_walk}time_{walk} $

$ $$V_{cycling}$$=ASC_{cycling}+\beta_{time\_cycle}time_{cycle} $

$ $$V_{public}$$=ASC_{public}+\beta_{time\_public}time_{public}+\beta_{cost}cost_{public}+\beta_{interchange}Number_{interchange}$

$ $$V_{driving}$$=ASC_{driving}+\beta_{time\_driving}time_{driving}+\beta_{cost}cost_{driving}+\beta_{drivingliscence}drivingliscence$

In [237]:
asc_walk = Beta('asc_walk', 0, None, None, 1) # Normalizing asc_walk to 0
asc_cycling = Beta('asc_cycling', 0, None, None, 0)
asc_public = Beta('asc_public', 0, None, None, 0)
asc_driving = Beta('asc_driving', 0, None, None, 0)

In [238]:
cost_driving = cost_driving_fuel + cost_driving_ccharge
dur_public = dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int # total duration of public transportation

In [239]:
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycling = Beta('beta_time_cycling', 0, None, None, 0)
beta_time_driving = Beta('beta_time_driving', 0, None, None, 0)
beta_time_public = Beta('beta_time_public', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_interchange = Beta('beta_interchange',0, None, None,0)
beta_drivingliscence = Beta('beta_drivingliscence',0,None,None,0)

### Define Model 2

In [240]:
V_walk = asc_walk + beta_time_walk * dur_walking
V_cycling = asc_cycling + beta_time_cycling * dur_cycling
V_public = asc_public + beta_time_public * dur_public + beta_cost * cost_transit + beta_interchange * pt_interchanges
V_driving = asc_driving + beta_time_driving * dur_driving + beta_cost * cost_driving + beta_drivingliscence * driving_license

V = {1: V_walk, 2: V_cycling, 3: V_public, 4: V_driving}

logprob = models.loglogit(V, None, travel_mode)
'''prob_walk = 1 / (1 + exp(V_cycling - V_walk) + exp(V_driving - V_walk) + exp(V_public - V_walk))
prob_cycling = 1 / (1 + exp(V_walk - V_cycling) + exp(V_driving - V_cycling) + exp(V_public - V_cycling))
prob_driving = 1 / (1 + exp(V_cycling - V_driving) + exp(V_walk - V_driving) + exp(V_public - V_driving))
prob_public = 1-prob_driving + prob_cycling + prob_walk

prob_observation = prob_walk * (travel_mode == 1) + prob_cycling * (travel_mode == 2) + prob_driving * (travel_mode == 4) + prob_public * (travel_mode == 3) 
logprob = log(prob_observation)'''

biogeme.modelName = 'model_2'

In [241]:
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'model_2'

In [242]:
results2 = biogeme.estimate()

In [243]:
print(results2.printGeneralStatistics())

Number of estimated parameters:	10
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4207.638
Final log likelihood:	-4207.638
Likelihood ratio test for the init. model:	2.745219e-07
Rho-square for the init. model:	3.26e-11
Rho-square-bar for the init. model:	-0.00238
Akaike Information Criterion:	8435.277
Bayesian Information Criterion:	8500.449
Final gradient norm:	2.9910E-02
Nbr of threads:	4



In [244]:
results2.getEstimatedParameters()

Value  Rob. Std err  Rob. t-test  Rob. p-value
asc_cycling          -4.657299      0.184990   -25.175912      0.000000
asc_driving          -2.924760      0.134271   -21.782439      0.000000
asc_public           -2.494754      0.125799   -19.831194      0.000000
beta_cost            -0.130478      0.013882    -9.398759      0.000000
beta_drivingliscence  1.359150      0.069274    19.619857      0.000000
beta_interchange     -0.030186      0.085146    -0.354520      0.722949
beta_time_cycling    -5.402394      0.443382   -12.184512      0.000000
beta_time_driving    -6.437378      0.374040   -17.210415      0.000000
beta_time_public     -3.393840      0.259014   -13.102899      0.000000
beta_time_walk       -8.563141      0.370435   -23.116463      0.000000

In [245]:
results2.data.htmlFileName

'model_2.html'

In [246]:
results2.likelihood_ratio_test(results1, alpha)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=408.7390215501109, threshold=5.991464547107979)

Model 2 is the preferred model